<a href="https://colab.research.google.com/github/ixomaxip/nasoc/blob/master/src/LaBSE_k_fold_dual_language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODO:
previous year data

In [1]:
!nvidia-smi

Sun Aug 22 15:48:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%capture

!pip install nltk emoji
!pip install tweet-preprocessor

!pip install transformers
!pip install transformers[sentencepiece]
!pip install datasets

!wget https://hasocfire.github.io/hasoc/2021/files/en_Hasoc2021_train.zip
!unzip -P hasoc2021_en en_Hasoc2021_train.zip
!ls


In [3]:
import os
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

import datetime
import re

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [39]:
#constants and parameters
seed = 42

path_gdrive_saved = '/content/drive/My Drive/nasoc/bert_tuned.weights'

batch_size = 8

DOWNLOAD_DATASET = False


In [ ]:
#@title
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#@title
# from google.colab import files
# uploaded = files.upload()

# !rm -rf /root/.ssh
# !mkdir /root/.ssh
# # !tar -xvzf ssh.tar.gz -C /root/.ssh && rm -rf ssh.tar.gz && chmod 700 /root/.ssh
# !tar -xvzf ssh.tar.gz -C /root/.ssh && rm -rf ssh.tar.gz
# !ssh-keyscan -t rsa github.com >> /root/.ssh/known_hosts

# !git config --global user.email 'ixomaxip@gmail.com'
# !git config --global user.user 'ixomaxip'
# !ssh -T git@github.com

# !git clone git@github.com:ixomaxip/nasoc.git

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Translate hindi DS

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("salesken/translation-hi-en")
model = AutoModelForSeq2SeqLM.from_pretrained("salesken/translation-hi-en")

Downloading:   0%|          | 0.00/235 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/813k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.18M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/302M [00:00<?, ?B/s]

In [ ]:
DEBUG = True
def translate(text):
    if DEBUG:
        return text
    hin_snippet = text #"कोविड के कारण हमने अपने ऋण ब्याज को कम कर दिया है"
    inputs = tokenizer.encode(hin_snippet, return_tensors="pt",padding=True,max_length=512,truncation=True)

    outputs = model.generate(inputs, max_length=128, num_beams=None, early_stopping=True)

    translated = tokenizer.decode(outputs[0]).replace('<pad>',"").strip().lower()
    return translated


path_to_hindi = '/content/drive/MyDrive/nasoc/hi_Hasoc2021_train.csv'

raw_df_hi = pd.read_csv(path_to_hindi)
raw_df_hi.head()
raw_df_hi['inputs'] = pd.Categorical(raw_df_hi['task_1'])

hi_text = []

index = 0
print(len(raw_df_hi.index))
for line in raw_df_hi.iterrows():
    index += 1
    if index == 0:
        continue # a lame attempt to skip the first row :)
    # if index % 100 == 0:
    #     print(index)
    hi_text.append((translate(line[1].text), line[1].task_1))

hi_df_tmp = pd.DataFrame(hi_text, columns=['text','inputs'])
train_hi_df, test_hi_df = train_test_split(hi_df_tmp, test_size=0.2, random_state=seed)

4594
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500


In [ ]:
print(train_hi_df)

                                                   text inputs
1402  #JusticeForShahabuddin #JusticeForShahabuddinR...    NOT
1736  ममता बैनर्जी शर्म करो, हिंसा आगज़नी का खेल बंद...    HOF
4438  वाकई अगर बंगाल में हिंदुओं को बचाना @narendram...    NOT
3680  @narendramodi @MamataOfficial जब 1947 में हिंद...    HOF
1025  बंगाल में हो रही हत्या,बलात्कार और लूट की जिम्...    NOT
...                                                 ...    ...
4426  @JPNadda Nadda Ji, बाजार से चूड़िंयां खरीद लीज...    HOF
466   रोजी, रोटी, भूख का डर हर किसी को होता हैं परन्...    NOT
3092  भक्त :- ऐसा #PM दोबारा नहीं मिलेगा   देश :- न ...    HOF
3772  कल से 18 से ऊपर वालो को हवा की वैक्सीन लगेगी.....    NOT
860   @AcharyaPramodk @yadavakhilesh अरे वाह जय हो ब...    HOF

[3675 rows x 2 columns]


# Custom BERTweet

In [75]:
# dataset

filename_en = 'en_Hasoc2021_train.csv'
filename_hi = '/content/drive/MyDrive/nasoc/hi_Hasoc2021_train.csv'
filename_mr = '/content/drive/MyDrive/nasoc/mr_Hasoc2021_train.csv'

def split_df(df):
    df['inputs'] = pd.Categorical(df['task_1'])
    df['inputs'] = df.inputs.cat.codes

    df['task_2_cat'] = pd.Categorical(df['task_2'])
    df['task_2_cat'] = df.task_2_cat.cat.codes

    train_df, test_df = train_test_split(df, test_size=0.2, random_state=seed)    
    return (train_df, test_df)

raw_df_en = pd.read_csv(filename_en)
#train_df_en_df, test_df_en_df = train_test_split(raw_df_en, test_size=0.2, random_state=seed)    # remove test data from raw_df_en

raw_df_hi = pd.read_csv(filename_hi)
train_df_hi_df, test_df_hi_df = train_test_split(raw_df_hi, test_size=0.2, random_state=seed)    # remove test data from raw_df_hi

raw_df_mr = pd.read_csv(filename_mr)

##raw_df = pd.concat([raw_df_en, train_df_hi_df, raw_df_mr], ignore_index=True) uncomment if you want to use all three datasets
raw_df = raw_df_hi.copy(deep=True)

# raw_df['inputs'] = pd.Categorical(raw_df['task_1'])
# raw_df['inputs'] = raw_df.inputs.cat.codes

# raw_df['task_2_cat'] = pd.Categorical(raw_df['task_2'])
# raw_df['task_2_cat'] = raw_df.task_2_cat.cat.codes

# train_df, test_df = train_test_split(raw_df, test_size=0.2, random_state=seed)
train_df, test_df = split_df(raw_df)

sr_train = train_df.groupby('task_1').count()['text']
sr_test  = test_df.groupby('task_1').count()['text']

tr_pr = sr_train['NOT']/sr_train['HOF']
ts_pr = sr_test['NOT']/sr_test['HOF']

#copied from https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

# copied from https://stackoverflow.com/questions/64719706/cleaning-twitter-data-pandas-python
def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    #tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI) #Remove Emojis
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    tweet = " ".join(tweet.split())
    tweet = remove_emojis(tweet)
    #tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) \
    #     if w.lower() in words or not w.isalpha())
    return tweet


raw_df_tmp = raw_df.copy(True)
#train_df, test_df = train_test_split(raw_df_tmp, test_size=0.2, random_state=seed)
train_df, test_df = split_df(raw_df_tmp)

#text = train_df.pop('text')#.map(lambda line: cleaner(line))
#label = train_df.pop('inputs')

#text_val = test_df.pop('text')#.map(lambda line: cleaner(line))
#label_val = test_df.pop('inputs')

#train_ds = tf.data.Dataset.from_tensor_slices((text.values, label.values))
#val_ds = tf.data.Dataset.from_tensor_slices((text_val.values, label_val.values))

#train_ds = train_ds.batch(batch_size=batch_size, drop_remainder=True)
#val_ds = val_ds.batch(batch_size=batch_size, drop_remainder=True)

train_df_en, test_df_en = split_df(raw_df_hi) # yes, we are using hindi

In [76]:
import time, datetime
import random
import numpy as np

import torch
from torch import nn
from torch.nn import functional as F, Linear
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import AutoTokenizer, AutoModel, AdamW
from transformers import get_linear_schedule_with_warmup

In [64]:
print(raw_df.text)

0       @wealth if you made it through this &amp;&amp;...
1       Technically that's still turning back the cloc...
2       @VMBJP @BJP4Bengal @BJP4India @narendramodi @J...
3       @krtoprak_yigit Soldier of Japan Who has dick ...
4       @blueheartedly You'd be better off asking who ...
                              ...                        
9387    आठवीत शिकत असलेल्या शिरीषकुमारने   हिंमत असेल ...
9388    तुझा पक्ष पण हिंदुत्ववादीच आहे ना? मग आणा की ब...
9389    च्यायला...२ महिने आधी हेच भिकारचोट केंद्राकडे ...
9390    १ मे १९६० ला स्थापन झाल्यावर राज्याचे सुसंघटित...
9391                               गीत भिमा तुझ्यासाठी ❣️
Name: text, Length: 9392, dtype: object


## Dataloader

In [77]:
class HasocDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, padding=True, truncation=True, max_length=64, return_tensors='pt')
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

MODEL_NAME = 'sentence-transformers/LaBSE'
#MODEL_NAME = "cardiffnlp/twitter-roberta-base-hate"
#MODEL_NAME = "vinai/bertweet-base"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
train_dataset = HasocDataset(train_df['text'].to_list(), train_df['inputs'].to_list(), tokenizer)
test_dataset = HasocDataset(test_df['text'].to_list(), test_df['inputs'].to_list(), tokenizer)

test_dataset_en = HasocDataset(test_df_en['text'].to_list(), test_df_en['inputs'].to_list(), tokenizer)

# train_hi_dataset = 
# test_hi_dataset = 

raw_dataset = HasocDataset(raw_df['text'].to_list(), raw_df['inputs'].to_list(), tokenizer)

BATCH_SIZE = 16

print(train_df.text.head())
print(test_df_en.text.head())

1402    #JusticeForShahabuddin #JusticeForShahabuddinR...
1736    ममता बैनर्जी शर्म करो, हिंसा आगज़नी का खेल बंद...
4438    वाकई अगर बंगाल में हिंदुओं को बचाना @narendram...
3680    @narendramodi @MamataOfficial जब 1947 में हिंद...
1025    बंगाल में हो रही हत्या,बलात्कार और लूट की जिम्...
Name: text, dtype: object
3201    #AAP  के विधायक ने  माना कि   #दिल्ली में जनता...
2820    ज़िंदगी और मौत दोनों से डर रहे है। बंदे में कु...
1616    अजीत सिंह के लिए आप का शोक व्यक्त करना बिल्कुल...
2696    @DhanBhalave @SNGupta82265724 @janardanspeaks ...
3487    लाशो पर लाशे गिरी तेरी नींद तक ना टूटी 😡😡😡 हाक...
Name: text, dtype: object


In [ ]:
# it = iter(true_test_dataloader)
# first = next(it)

# print(first)

## Customization class & auxiliary functions

In [78]:
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# DEVICE = torch.device('cpu')

# Customization
class CustomBERTweet(nn.Module):
    def __init__(self):
        super(CustomBERTweet, self).__init__()
        self.bertweet = AutoModel.from_pretrained(MODEL_NAME)
        #self.cnn = nn.Conv1d(1, 768, 3)
        self.dropout = nn.Dropout(0.1)
        self.dense1 = nn.Linear(768, 2)
        # self.dense2 = nn.Linear(256, 2)
    
    def forward(self, feed_dict):

        bertweet_output = self.bertweet(**feed_dict)
        drop_output = self.dropout(bertweet_output.pooler_output)
        linear1_output = self.dense1(drop_output)
        # linear2_output = self.dense2(linear1_output)

        return linear1_output

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

def forward(batch, model):
    # feed_dict = {
    #     'input_ids': batch['input_ids'].to(DEVICE),
    #     'token_type_ids': batch['token_type_ids'].to(DEVICE),
    #     'attention_mask': batch['attention_mask'].to(DEVICE),
    # }
    feed_dict = {
        'input_ids': batch['input_ids'].to(DEVICE),
        #'token_type_ids': batch['token_type_ids'].to(DEVICE),
        'attention_mask': batch['attention_mask'].to(DEVICE),
    }    
    outputs = model(feed_dict)
    outputs = F.log_softmax(outputs, dim=1)

    return outputs

def validate(model, dataloader, as_dict=False):
    predictions, targets = [], []
    for batch in dataloader:
        target = batch['labels']
        with torch.no_grad():
            logits = forward(batch, model)
        logits = logits.detach().cpu().numpy()
        label = target.to('cpu').numpy()        
        predictions.append(logits)
        targets.append(label)
    flat_predictions = [item for sublist in predictions for item in sublist]
    flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
    flat_targets = [item for sublist in targets for item in sublist]
    return classification_report(flat_targets, flat_predictions, output_dict=as_dict)

def reset_weights(m):
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            #print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

In [ ]:
# del model

## Create model

In [79]:
true_train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = BATCH_SIZE
)
true_test_dataloader = DataLoader(
            test_dataset,
            sampler = RandomSampler(test_dataset),
            batch_size = BATCH_SIZE
)

the_en_test_dataset = DataLoader(
            test_dataset_en,
            sampler = RandomSampler(test_dataset_en),
            batch_size = BATCH_SIZE
)

len(true_train_dataloader)

#model = CustomBERTweet()

230

## Check model

In [ ]:
# Get all of the model's parameters as a list of tuples.
# params = list(model.named_parameters())
# print(f'The BERT model has {len(params):} different named parameters.\n')
# print('==== Embedding Layer ====\n')
# for p in params[0:5]:
#     print(f'{p[0]:<55} {str(tuple(p[1].size())):>12}')
# print('\n==== First Transformer ====\n')
# for p in params[5:21]:
#     print(f'{p[0]:<55} {str(tuple(p[1].size())):>12}')
# print('\n==== Output Layer ====\n')
# for p in params[-12:]:
#     print(f'{p[0]:<55} {str(tuple(p[1].size())):>12}')

In [ ]:
# model.to(torch.device('cuda'))
# print(validate(model, test_dataloader))

## Train

In [80]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GroupKFold

k_folds = 5
EPOCHS = 4

# For fold results
results = {}

kfold = KFold(n_splits=k_folds, shuffle=True)


In [81]:
# # model = CustomBERTweet()
# # params = list(model.named_parameters())
# # model.to(torch.device('cuda'))
# # print(validate(model, test_dataloader))
fold_results = []

for fold, (train_ids_idx, test_ids_idx) in enumerate(kfold.split(train_dataset)):    
    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
#     # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids_idx)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids_idx)

    # Define data loaders for training and testing data in this fold
    k_fold_train_dataloader = DataLoader(
            train_dataset,
            sampler = train_subsampler,
            batch_size = BATCH_SIZE
    )
    k_fold_test_dataloader = DataLoader(
            train_dataset,
            sampler = test_subsampler,
            batch_size = BATCH_SIZE
    )

    model_kf = CustomBERTweet()

    total_steps = len(k_fold_train_dataloader) * EPOCHS
    optimizer = AdamW(model_kf.parameters(), lr = 2e-5, eps = 1e-8)
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 20,
                                                num_training_steps = total_steps)
    loss_func = nn.CrossEntropyLoss()
    model_kf.to(DEVICE)

    #print(validate(model_kf, test_dataloader))

    start = time.time()
    for epoch in range(0, EPOCHS):
        total_train_loss = 0
        model_kf.train()
        print(f'======== Epoch {epoch+1} / {EPOCHS} ========')
        for step, batch in enumerate(k_fold_train_dataloader):
            #print(f"step: {step}")
            if step % 50 == 0 and not step == 0:
                elapsed = format_time(time.time() - start)
                print(f'\tBatch {step:>5,}  of  {len(k_fold_train_dataloader):>5,}.    Elapsed: {elapsed}.')
                # print(f'\tloss: {loss.item()}')
            optimizer.zero_grad()
            outputs = forward(batch, model_kf)
            target = batch['labels'].to(DEVICE)
            model_kf.zero_grad()
            loss = loss_func(outputs, target)
            total_train_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model_kf.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
        
        avg_train_loss = total_train_loss / len(k_fold_test_dataloader)
        training_time = format_time(time.time() - start)
        print(f'\tAverage training loss: {avg_train_loss:.2f}')
        print(f'\tTraining epoch took: {training_time}')
        model_kf.eval()
        report = validate(model_kf, k_fold_test_dataloader)
        print(f'======== Validation report {epoch+1} / {EPOCHS} ========')
        print(report)

    avg_train_loss = total_train_loss / len(the_en_test_dataset) #len(true_test_dataloader)
    training_time = format_time(time.time() - start)
    model_kf.eval()
    report = validate(model_kf, the_en_test_dataset, True) # true_test_dataloader, True)
    fold_results.append(report)
    print(f'======== fold {fold} validation ========')
    #print(report)
    df = pd.DataFrame(report)
    print(df.T)

print(report)
    

FOLD 0
--------------------------------
======== Epoch 1 / 4 ========


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


	Batch    50  of    184.    Elapsed: 0:00:10.
	Batch   100  of    184.    Elapsed: 0:00:19.
	Batch   150  of    184.    Elapsed: 0:00:29.
	Average training loss: 1.95
	Training epoch took: 0:00:36
======== Validation report 1 / 4 ========
              precision    recall  f1-score   support

           0       0.68      0.67      0.67       219
           1       0.86      0.87      0.86       516

    accuracy                           0.81       735
   macro avg       0.77      0.77      0.77       735
weighted avg       0.81      0.81      0.81       735

======== Epoch 2 / 4 ========
	Batch    50  of    184.    Elapsed: 0:00:47.
	Batch   100  of    184.    Elapsed: 0:00:57.
	Batch   150  of    184.    Elapsed: 0:01:06.
	Average training loss: 1.38
	Training epoch took: 0:01:13
======== Validation report 2 / 4 ========
              precision    recall  f1-score   support

           0       0.75      0.65      0.69       219
           1       0.86      0.91      0.88       516

 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


	Batch    50  of    184.    Elapsed: 0:00:10.
	Batch   100  of    184.    Elapsed: 0:00:19.
	Batch   150  of    184.    Elapsed: 0:00:29.
	Average training loss: 1.95
	Training epoch took: 0:00:36
======== Validation report 1 / 4 ========
              precision    recall  f1-score   support

           0       0.67      0.71      0.69       212
           1       0.88      0.85      0.87       523

    accuracy                           0.81       735
   macro avg       0.77      0.78      0.78       735
weighted avg       0.82      0.81      0.82       735

======== Epoch 2 / 4 ========
	Batch    50  of    184.    Elapsed: 0:00:47.
	Batch   100  of    184.    Elapsed: 0:00:57.
	Batch   150  of    184.    Elapsed: 0:01:06.
	Average training loss: 1.45
	Training epoch took: 0:01:13
======== Validation report 2 / 4 ========
              precision    recall  f1-score   support

           0       0.69      0.74      0.71       212
           1       0.89      0.86      0.88       523

 

...

	Batch    50  of    184.    Elapsed: 0:00:10.
	Batch   100  of    184.    Elapsed: 0:00:19.
	Batch   150  of    184.    Elapsed: 0:00:29.
	Average training loss: 1.96
	Training epoch took: 0:00:36
======== Validation report 1 / 4 ========
              precision    recall  f1-score   support

           0       0.85      0.49      0.62       246
           1       0.79      0.96      0.86       489

    accuracy                           0.80       735
   macro avg       0.82      0.72      0.74       735
weighted avg       0.81      0.80      0.78       735

======== Epoch 2 / 4 ========
	Batch    50  of    184.    Elapsed: 0:00:47.
	Batch   100  of    184.    Elapsed: 0:00:57.
	Batch   150  of    184.    Elapsed: 0:01:06.
	Average training loss: 1.44
	Training epoch took: 0:01:13
======== Validation report 2 / 4 ========
              precision    recall  f1-score   support

           0       0.74      0.61      0.67       246
           1       0.82      0.89      0.85       489

 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


	Batch    50  of    184.    Elapsed: 0:00:10.
	Batch   100  of    184.    Elapsed: 0:00:19.
	Batch   150  of    184.    Elapsed: 0:00:29.
	Average training loss: 1.94
	Training epoch took: 0:00:36
======== Validation report 1 / 4 ========
              precision    recall  f1-score   support

           0       0.65      0.65      0.65       228
           1       0.84      0.84      0.84       507

    accuracy                           0.78       735
   macro avg       0.75      0.75      0.75       735
weighted avg       0.78      0.78      0.78       735

======== Epoch 2 / 4 ========
	Batch    50  of    184.    Elapsed: 0:00:47.
	Batch   100  of    184.    Elapsed: 0:00:57.
	Batch   150  of    184.    Elapsed: 0:01:06.
	Average training loss: 1.37
	Training epoch took: 0:01:13
======== Validation report 2 / 4 ========
              precision    recall  f1-score   support

           0       0.69      0.68      0.69       228
           1       0.86      0.86      0.86       507

 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


	Batch    50  of    184.    Elapsed: 0:00:10.
	Batch   100  of    184.    Elapsed: 0:00:19.
	Batch   150  of    184.    Elapsed: 0:00:29.
	Average training loss: 1.97
	Training epoch took: 0:00:36
======== Validation report 1 / 4 ========
              precision    recall  f1-score   support

           0       0.76      0.58      0.66       231
           1       0.83      0.92      0.87       504

    accuracy                           0.81       735
   macro avg       0.79      0.75      0.76       735
weighted avg       0.81      0.81      0.80       735

======== Epoch 2 / 4 ========
	Batch    50  of    184.    Elapsed: 0:00:47.
	Batch   100  of    184.    Elapsed: 0:00:57.
	Batch   150  of    184.    Elapsed: 0:01:06.
	Average training loss: 1.36
	Training epoch took: 0:01:13
======== Validation report 2 / 4 ========
              precision    recall  f1-score   support

           0       0.69      0.70      0.69       231
           1       0.86      0.86      0.86       504

 

In [82]:
# Calc summary, i.e. k-folds avegare
class_avg = {}

if len(fold_results) > 0:
    class_avg = fold_results[0]

counter = 0

# sum
for fold_report in fold_results:
    if counter == 0:
        counter += 1
        continue
    
    for key, value in fold_report.items(): #0, 1, accuracy...
        if type(class_avg[key]) is float:
            class_avg[key] += value
        else:
            for key_metr, value_metr in value.items():
                class_avg[key][key_metr] += value_metr
                
    counter += 1

# div
for key, value in fold_report.items(): #0, 1, accuracy...
    if type(class_avg[key]) is float:
        class_avg[key] /= counter
    else:
        for key_metr, value_metr in value.items():
            class_avg[key][key_metr] /= counter

#print
df = pd.DataFrame(class_avg)
print(df.T)

              precision    recall  f1-score     support
0              0.708376  0.659933  0.682892  297.000000
1              0.842767  0.869775  0.855966  622.000000
accuracy       0.801959  0.801959  0.801959    0.801959
macro avg      0.775572  0.764854  0.769429  919.000000
weighted avg   0.799335  0.801959  0.800032  919.000000


#Garbage

In [ ]:
from transformers import Trainer, TrainingArguments
OUTPUT_DIR = os.path.join(ROOT_DIR, 'labse_ft_results')

training_args = TrainingArguments(
    output_dir = OUTPUT_DIR,          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=0,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir=os.path.join(OUTPUT_DIR, 'logs'),            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=custom_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)
trainer.train()

***** Running training *****
  Num examples = 3074
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 579
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


TypeError: ignored

In [ ]:
max_len = 64
row = train_df.iloc[0]
texts = row.text
encoded_input = tokenizer([texts, texts], padding=True, truncation=True, max_length=max_len, return_tensors='pt').to(device=DEVICE)
with torch.no_grad():
    model_output = model(**encoded_input)
embeddings = model_output.pooler_output
embeddings = torch.nn.functional.normalize(embeddings)

In [ ]:
model_output.pooler_output.shape

torch.Size([2, 768])

In [ ]:
train_embedings = text_encoder(train_texts, model, tokenizer, 64)

In [ ]:
sentences = ["Hello World"]
encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=max_len, return_tensors='pt').to(DEVICE)
with torch.no_grad():
    model_output = model(**encoded_input)
embeddings = model_output.pooler_output
embeddings = torch.nn.functional.normalize(embeddings)
# print(embeddings[0])

In [ ]:
model_output.pooler_output.shape

torch.Size([1, 768])

In [ ]:
model.parameters()

<generator object Module.parameters at 0x7f24480df4d0>

In [ ]:
len(train_labels)

3074

In [ ]:


train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)